In [2]:
import pandas as pd

data_file_path = "./datasets/generated/cleaned_aggregated_data.json"

df = pd.read_json(data_file_path)

df.head()

,tmID,year,playoff,averageWinRate,averagePoints,averageRebounds,averageAssists,averageSteals,averageBlocks,averageTurnovers,averageFGRatio,averageFTRatio,averageThreeRatio,coachWinRate,numberOfAwardedPlayers
0,WAS,10,Y,0.294118,189.512143,84.572143,42.982857,19.696786,7.073929,44.290357,0.408655,0.717564,0.306577,0.500000,0
1,WAS,9,N,0.470588,189.708943,82.593336,36.207412,20.147909,8.820809,38.090105,0.402735,0.800336,0.276818,0.500000,2
2,WAS,8,N,0.529412,239.624444,95.275556,50.680741,25.660000,10.042222,44.559259,0.434105,0.760861,0.303131,0.529412,2
3,WAS,7,Y,0.470588,216.456929,88.321161,42.381086,23.959738,8.204120,39.441011,0.422553,0.743379,0.337256,0.470588,2
4,WAS,6,N,0.500000,193.306310,92.482727,39.100875,21.625518,11.273146,39.394288,0.429846,0.706172,0.334885,0.437500,1
